# UrbanSoundsClassification: AIYard

Model.ipynb

AIYard ekibi

### 1. Kütüphanelerin içe aktarılması

In [2]:
import os
import numpy as np
import tensorflow as tf

### 2. Önişlenmiş verisetlerinin içe aktarılması
Önişleme defterini kullanarak `preproc` klasörü altında oluşturduğumuz `train`, `val` ve `test` klasörlerindeki spektrogramları okuyup demetler halinde kaydederek başlıyoruz.

Google Colab ortamında isek Google Drive'ı mount ettikten sonra `preproc` klasörüne giden yolu tutan `preproc_folder`'ı değiştirmeliyiz.

In [3]:
from google.colab import drive
drive.mount('/content/GDrive/')

Mounted at /content/GDrive/


Lokal cihazımda önişleyip shuffle ederek oluşturduğum spektrogramlara https://drive.google.com/file/d/133_6nrhE7261WkFMjbHxAQSGWa7z0Mmq/view?usp=sharing linkinden erişebilirsiniz. Burada Google Drive'dan session hafızasına alıp untar ediyoruz ve `preproc_folder` değişkenine `/content/preproc/` yolunu veriyoruz. 

In [5]:
!cp /content/GDrive/MyDrive/preproc.tar /content
!tar -xvf preproc.tar

Streaming output truncated to the last 5000 lines.
preproc/train/spec-2720-4.npy
preproc/train/spec-5198-7.npy
preproc/train/spec-2587-8.npy
preproc/train/spec-647-9.npy
preproc/train/spec-6411-7.npy
preproc/train/spec-1929-7.npy
preproc/train/spec-2428-2.npy
preproc/train/spec-5881-3.npy
preproc/train/spec-5069-4.npy
preproc/train/spec-5649-7.npy
preproc/train/spec-4136-5.npy
preproc/train/spec-5946-8.npy
preproc/train/spec-1915-2.npy
preproc/train/spec-1325-5.npy
preproc/train/spec-12-9.npy
preproc/train/spec-4281-8.npy
preproc/train/spec-5815-4.npy
preproc/train/spec-3260-8.npy
preproc/train/spec-262-0.npy
preproc/train/spec-934-3.npy
preproc/train/spec-3686-6.npy
preproc/train/spec-4152-4.npy
preproc/train/spec-58-3.npy
preproc/train/spec-3533-9.npy
preproc/train/spec-3354-5.npy
preproc/train/spec-475-8.npy
preproc/train/spec-3805-0.npy
preproc/train/spec-2301-3.npy
preproc/train/spec-4016-9.npy
preproc/train/spec-3555-3.npy
preproc/train/spec-372-0.npy
preproc/train/spec-2220-4.np

In [6]:
# DİKKAT: Yolun sonuna "/" karakteri eklemeyi unutma!
# /home/ferit/repos/UrbanSoundsClassification/preproc/
# /content/preproc/
preproc_folder = "/content/preproc/"

#### 2.0. Dosya okuma fonksiyonu
Spektrogramları okumak için argüman olarak klasörün yolunu alan ve geriye spektrogram ile sınıf numarasını dönen fonksiyon yazacağız.

In [10]:
def read_spectrograms(folder_path: str):
  X_list = []
  Y_list = []
  for path, subdirs, files in os.walk(folder_path):
    for name in files:
      file = os.path.join(path, name)
      X_list.append(np.load(file))
      Y_list.append(int(file.split('-')[2][0]))
  return X_list, Y_list

#### 2.1. `train` klasöründeki spektrogramların okunması

X_train ve Y_train listelerini elde edeceğiz. Her iki listede 6900 tane eleman bulunmalı. 

X_train'e spektrogram verisi olan (64, 64) boyutlu NumPy dizilerini, Y_train'e ise karşılık gelen sınıfın numarasını ekleyeceğiz.

In [11]:
X_train, Y_train = read_spectrograms(f"{preproc_folder}train")

In [12]:
len(X_train) == len(Y_train) == 6900

True

#### 2.2. `val` klasöründeki spektrogramların okunması

X_val ve Y_val listelerini elde edeceğiz. Her iki listede 900 tane eleman bulunmalı.

In [13]:
X_val, Y_val = read_spectrograms(f"{preproc_folder}val")

In [14]:
len(X_val) == len(Y_val) == 900

True

#### 2.3. `test` klasöründeki spektrogramların okunması

X_test ve Y_test listelerini elde edeceğiz. Her iki listede 935 tane eleman bulunmalı.

In [15]:
X_test, Y_test = read_spectrograms(f"{preproc_folder}test")

In [16]:
len(X_test) == len(Y_test) == 935

False

### 3. Nöral ağın oluşturulması